<!--NOTEBOOK_HEADER-->
*This notebook contains material from the [ND-Pyomo-Cookbook](https://jckantor.github.io/ND-Pyomo-Cookbook) by
Jeffrey Kantor (jeff at nd.edu); the content is available [on Github](https://github.com/jckantor/ND-Pyomo-Cookbook.git).
*The text is released under the [CC-BY-NC-ND-4.0 license](https://creativecommons.org/licenses/by-nc-nd/4.0/legalcode),
and code is released under the [MIT license](https://opensource.org/licenses/MIT).*

<!--NAVIGATION-->
< [Installing a Pyomo/Python Development Environment](http://nbviewer.jupyter.org/github/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/01.01-Installing-Pyomo.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Running Pyomo on the Notre Dame CRC Cluster](http://nbviewer.jupyter.org/github/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/01.03-Running-Pyomo-on-the-Notre-Dame-CRC-Cluster.ipynb) ><p><a href="https://colab.research.google.com/github/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/01.02-Running-Pyomo-on-Google-Colab.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Running Pyomo on Google Colab

Keywords: installation

This note notebook shows how to install the basic pyomo package on Google Colab, and then demonstrates the subsequent installation and use of various solvers including

* GLPK
* COIN-OR CBC
* COIN-OR Ipopt
* COIN-OR Bonmin
* COIN-OR Couenne
* COIN-OR Gecode

## Basic installation of Pyomo

We'll do a quiet installation of pyomo using `pip`.  This needs to be done once at the start of each Colab session.

The installation of pyomo can be verified by entering a simple model. We'll use the model again in subsequent cells to demonstrate the installation and execution of various solvers.

In [4]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(expr = 40*model.x + 30*model.y, sense=maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x + 30*y

3 Constraint Declarations
    demand : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :    x :  40.0 :   True
    laborA : Size=1, Index=None, Active=True
        Key  : Lower : Body  : Upper : Active
        None :  -Inf : x + y :  80.0 :   True
    laborB : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y : 100.0 :   True

6 Declarations: x y profit demand laborA laborB


In [5]:
import math
from pyomo.environ import ConcreteModel, Var, Objective, NonNegativeReals, Constraint, Suffix, exp, value
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np
import pickle
#import pickle5 as pickle
import matplotlib.pyplot as plt

import time
startTime = time.time()
plt.style.use("seaborn")


#%% Scenarios and settings

# scenario = "no_co2-no_learning"
# scenario = "co2-0p2-no_learning"
scenario = "co2-0p2-learning"
# scenario = "no_co2-learning"

# learning_scenario = "high_learning"
# learning_scenario = "low_learning"
learning_scenario = "nom_learning"


# CO2 budget for 2050 global warming goals
co2_until_2050 = 1e10 # 100 million tCO2 ,10000000000 # 10 gigaton CO2

# Greenfield scenario 
Greenfield = True


# legend on/off when plotting
lgnd = True

r = 0.00 # discount rate

hours = list(range(111))


parameters  = pd.read_pickle("parameters.pkl")
store_param = pd.read_pickle("store_param.pkl")
# demand = pd.DataFrame(columns= ["demand"])
demand = pd.read_pickle("df_elec.pkl")
demand.reset_index()
demand.drop(columns=["utc_time"])
demand = demand/1000

# CF_solar_one    = pd.read_pickle("CF_solar_one.pkl")
# CF_onwind_one   = pd.read_pickle("CF_onwind_one.pkl")
# CF_offwind_one  = pd.read_pickle("CF_offwind_one.pkl")

# Cf_solar    = pd.read_pickle("Cf_solar.pkl")
# Cf_onshore  = pd.read_pickle("Cf_onshore.pkl")
# Cf_offshore = pd.read_pickle("Cf_offshore.pkl")


Cf_solar      = pd.read_pickle("cf_solar3h.pkl")
Cf_onshore    = pd.read_pickle("cf_onshore3h.pkl")
Cf_offshore   = pd.read_pickle("cf_offshore3h.pkl")
demand2w3h      = pd.read_pickle("demand2w3h.pkl")
demand2w3h.reset_index()
demand2w3h = demand2w3h/1000



# parameters.at["current capital cost","coal"] = 10000

# demand = 400 #GWh

techs_file          = "techs.pkl"
fossil_techs_file   = "fossil_techs.pkl"
renewables_file     = "renewables.pkl"
wind_file           = "wind.pkl"
colors_file         = "colors.pkl"
storage_file        = "storage.pkl"
color_storage_file  = "color_storage.pkl"

files = [techs_file,fossil_techs_file,renewables_file,wind_file,colors_file,storage_file,color_storage_file]
lists = ["techs","fossil_techs","renewables","wind","colors","storage","color_storage"]


for i in range(len(files)):
    open_file = open(files[i], "rb")
    lists[i] = pickle.load(open_file)
    open_file.close()

techs           = lists[0]
fossil_techs    = lists[1]
renewables      = lists[2]
wind            = lists[3]
colors          = lists[4]
storage         = lists[5]
color_storage   = lists[6]


# Green- or brownfield scenario:
if Greenfield is True:
    for tech in techs:
        parameters.loc["existing age"]      = [0,0,0,0,0,0,0] #years
        parameters.loc["existing capacity"] = [0,0,0,0,0,0,0]
    print("Greenfield approach")
else:
    print("Brownfield approach")
    
# parameters.at["current capital cost","coal"] *= 0.5

#%% Updating learning rates and CO2 budget

#Currently installed capacities in GW is used to assume current demand

# Hourly demand
years = [2020,2023,2026,2029,2032,2035,2038,2041,2044,2047,2050]

# for year in range(7):
#     if year > 2020:
#         for i in demand:
#             demand.at[year,i] = 8+demand.at[year-1,i]
#     else:
#         for i in demand:
#             demand.at[year,i] = (600) #from EU Energy Outlook 2050


if "no_learning" in scenario:
    parameters.loc["learning rate"]     = 0
    store_param.loc["learning rate"]    = 0
    print("No learning")
else:
    if "high_learning" in learning_scenario:
        parameters.loc["learning rate"]     = [0.12,0.12,0.23,0.14,0.15,0.05,0.06] # [0.19,0.32,0.47,0.34,0.15,0.083]
        store_param.loc["learning rate"]    = [0.18,0.1,0.1,0.26,0.21]
        print("High learning rates")
    else: 
        if "low_learning" in learning_scenario:
            parameters.loc["learning rate"]     = [0.05,0,0.1,0,0.15,0.06,0.0] # [0.05,0,0.1,-0.01,0.15,0.06,-0.06]
            store_param.loc["learning rate"]    = [0.08,0.1,0.1,0.18,0.15]
            print("Low learning rates")
        else:
            # nom learning
            parameters.loc["learning rate"]     = [0.12,0.12,0.23,0.14,0.15,0.083,0.0] # [0.05,0,0.1,-0.01,0.15,0.06,-0.06]
            store_param.loc["learning rate"]    = [0.08,0.1,0.1,0.18,0.15]
            print("Nominal learning rates")


# Calculating learning parameter gamma
for i in range(len(techs)):
    parameters.loc["learning parameter"][i] = math.log(1/(1-parameters.loc["learning rate"][i])) / math.log(2)
for i in range(len(storage)):
    store_param.loc["learning parameter"][i] = math.log(1/(1-store_param.loc["learning rate"][i])) / math.log(2)


# carbon budget in average tCO2   
if "no_co2" in scenario:
    co2_budget = 1e30
    print("No CO2 budget")
else:
    co2_budget = co2_until_2050 # [tCO2] 10 Gigatons CO2
    print("CO2 budget of "+ str(co2_until_2050) + " tons CO2")




Greenfield approach
Nominal learning rates
CO2 budget of 10000000000.0 tons CO2


## pyomo model

In [7]:
#%%
# MWh_total = demand['demand'].sum()*1000*8760/50

    
#%% One node model
model = ConcreteModel()
model.generators            = Var(techs, years, within=NonNegativeReals)
model.generators_dispatch   = Var(techs, years, hours, within=NonNegativeReals)
model.generators_built      = Var(techs, years, within=NonNegativeReals)
model.fixed_costs           = Var(techs, years, within=NonNegativeReals)


model.storage               = Var(storage, years, within=NonNegativeReals)
model.stored_energy         = Var(storage, years, hours, within=NonNegativeReals)
model.storage_built         = Var(storage, years, within=NonNegativeReals)
model.fixed_costs_storage   = Var(storage, years, within=NonNegativeReals)
model.storage_charge        = Var(storage, years, hours, within=NonNegativeReals)
model.storage_discharge     = Var(storage, years, hours, within=NonNegativeReals)



#Value of currently installed technology:
constant = sum(parameters.at['existing capacity',tech] * parameters.at['current capital cost', tech]*1000/1e9/(1+r)**(hour-hours[0]) for tech in techs for hour in hours if hour < hours[0] + parameters.at['lifetime',tech] - parameters.at['existing age',tech])
print("Cost of existing capacity =", "%.2f"%constant)


model.objective = Objective(expr=constant +
                            sum(model.generators_built[tech,year] * model.fixed_costs[tech,year]/1e9 * sum(1/(1+r)**(yearb-years[0]) for yearb in years if ((yearb>=year) and (yearb < year + parameters.at['lifetime',tech])))
                              for tech in techs for year in years)+
                            sum(model.generators_dispatch[tech,2020,hour] * parameters.at['marginal cost',tech]*1000*26*3 / 1e9 /(1+r)**(2020-years[0])
                                for hour in hours
                                for tech in techs)+
                            sum(model.generators_dispatch[tech,2023,hour] * parameters.at['marginal cost',tech]*1000*26*3 / 1e9 /(1+r)**(2023-years[0])
                                for hour in hours
                                for tech in techs)+
                            sum(model.generators_dispatch[tech,2026,hour] * parameters.at['marginal cost',tech]*1000*26*3 / 1e9 /(1+r)**(2026-years[0])
                                for hour in hours
                                for tech in techs)+
                            sum(model.generators_dispatch[tech,2029,hour] * parameters.at['marginal cost',tech]*1000*26*3 / 1e9 /(1+r)**(2029-years[0])
                                for hour in hours
                                for tech in techs)+
                            sum(model.generators_dispatch[tech,2032,hour] * parameters.at['marginal cost',tech]*1000*26*3 / 1e9 /(1+r)**(2032-years[0])
                                for hour in hours
                                for tech in techs)+
                            sum(model.generators_dispatch[tech,2035,hour] * parameters.at['marginal cost',tech]*1000*26*3 / 1e9 /(1+r)**(2035-years[0])
                                for hour in hours
                                for tech in techs)+
                            sum(model.generators_dispatch[tech,2038,hour] * parameters.at['marginal cost',tech]*1000*26*3 / 1e9 /(1+r)**(2038-years[0])
                                for hour in hours
                                for tech in techs)+
                            sum(model.generators_dispatch[tech,2041,hour] * parameters.at['marginal cost',tech]*1000*26*3 / 1e9 /(1+r)**(2041-years[0])
                                for hour in hours
                                for tech in techs)+
                            sum(model.generators_dispatch[tech,2044,hour] * parameters.at['marginal cost',tech]*1000*26*3 / 1e9 /(1+r)**(2044-years[0])
                                for hour in hours
                                for tech in techs)+ 
                            sum(model.generators_dispatch[tech,2047,hour] * parameters.at['marginal cost',tech]*1000*26*3 / 1e9 /(1+r)**(2047-years[0])
                                for hour in hours
                                for tech in techs)+
                            sum(model.generators_dispatch[tech,2050,hour] * parameters.at['marginal cost',tech]*1000*26*3 / 1e9 /(1+r)**(2050-years[0])
                                for hour in hours
                                for tech in techs)+
                            sum(model.storage_built[tech,year] * model.fixed_costs_storage[tech,year]/1e9 * sum(1/(1+r)**(yearb-years[0]) for yearb in years if ((yearb>=year) and (yearb < year + store_param.at['lifetime',tech])))
                              for tech in storage for year in years))


#%% Balance Constraints

def balance_constraint2020(model,hour): # GWh
    return demand2w3h.at[hour,0]  <= sum(model.generators_dispatch[tech, 2020, hour] for tech in techs)  + model.storage_discharge['battery_store',2020,hour]
model.balance_constraint2020 = Constraint(hours, rule=balance_constraint2020)

def balance_constraint2023(model,hour):
    return demand2w3h.at[hour,0]  <= sum(model.generators_dispatch[tech, 2023, hour] for tech in techs)  + model.storage_discharge['battery_store',2023,hour]
model.balance_constraint2023 = Constraint( hours, rule=balance_constraint2023)

def balance_constraint2026(model,hour):
    return demand2w3h.at[hour,0]  <= sum(model.generators_dispatch[tech, 2026, hour] for tech in techs)  + model.storage_discharge['battery_store',2026,hour]
model.balance_constraint2026 = Constraint(hours, rule=balance_constraint2026)

def balance_constraint2029(model,hour):
    return demand2w3h.at[hour,0]  <= sum(model.generators_dispatch[tech, 2029, hour] for tech in techs) + model.storage_discharge['battery_store',2029,hour]
model.balance_constraint2029 = Constraint(hours, rule=balance_constraint2029)

def balance_constraint2032(model,hour):
    return demand2w3h.at[hour,0]  <= sum(model.generators_dispatch[tech, 2032, hour] for tech in techs) + model.storage_discharge['battery_store',2032,hour]
model.balance_constraint2032 = Constraint(hours, rule=balance_constraint2032)

def balance_constraint2035(model,hour):
    return demand2w3h.at[hour,0]  <= sum(model.generators_dispatch[tech, 2035, hour] for tech in techs) + model.storage_discharge['battery_store',2035,hour]
model.balance_constraint2035 = Constraint(hours, rule=balance_constraint2035)

def balance_constraint2038(model,hour):
    return demand2w3h.at[hour,0]  <= sum(model.generators_dispatch[tech, 2038, hour] for tech in techs) + model.storage_discharge['battery_store',2038,hour]
model.balance_constraint2038 = Constraint(hours, rule=balance_constraint2038)

def balance_constraint2041(model,hour):
    return demand2w3h.at[hour,0]  <= sum(model.generators_dispatch[tech, 2041, hour] for tech in techs) + model.storage_discharge['battery_store',2041,hour]
model.balance_constraint2041 = Constraint(hours, rule=balance_constraint2041)

def balance_constraint2044(model,hour):
    return demand2w3h.at[hour,0]  <= sum(model.generators_dispatch[tech, 2044, hour] for tech in techs) + model.storage_discharge['battery_store',2044,hour]
model.balance_constraint2044 = Constraint(hours, rule=balance_constraint2044)

def balance_constraint2047(model,hour):
    return demand2w3h.at[hour,0]  <= sum(model.generators_dispatch[tech, 2047, hour] for tech in techs) + model.storage_discharge['battery_store',2047,hour]
model.balance_constraint2047 = Constraint(hours, rule=balance_constraint2047)

def balance_constraint2050(model,hour):
    return demand2w3h.at[hour,0]  <= sum(model.generators_dispatch[tech, 2050, hour] for tech in techs) + model.storage_discharge['battery_store',2050,hour]
model.balance_constraint2050 = Constraint(hours, rule=balance_constraint2050)


#%% Solar capacity constraints

def solar_constraint2020(model,year,hour):
    return model.generators_dispatch["solar_PV",2020,hour] <= model.generators["solar_PV",2020]*Cf_solar.at[hour,0]
model.solar_constraint2020 = Constraint(years, hours, rule=solar_constraint2020)

def solar_constraint2023(model,year,hour):
    return model.generators_dispatch["solar_PV",2023,hour] <= model.generators["solar_PV",2023]*Cf_solar.at[hour,0]
model.solar_constraint2023 = Constraint(years, hours, rule=solar_constraint2023)

def solar_constraint2026(model,year,hour):
    return model.generators_dispatch["solar_PV",2026,hour] <= model.generators["solar_PV",2026]*Cf_solar.at[hour,0]
model.solar_constraint2026 = Constraint(years, hours, rule=solar_constraint2026)

def solar_constraint2029(model,year,hour):
    return model.generators_dispatch["solar_PV",2029,hour] <= model.generators["solar_PV",2029]*Cf_solar.at[hour,0]
model.solar_constraint2029 = Constraint(years, hours, rule=solar_constraint2029)

def solar_constraint2032(model,year,hour):
    return model.generators_dispatch["solar_PV",2032,hour] <= model.generators["solar_PV",2032]*Cf_solar.at[hour,0]
model.solar_constraint2032 = Constraint(years, hours, rule=solar_constraint2032)

def solar_constraint2035(model,year,hour):
    return model.generators_dispatch["solar_PV",2035,hour] <= model.generators["solar_PV",2035]*Cf_solar.at[hour,0]
model.solar_constraint2035 = Constraint(years, hours, rule=solar_constraint2035)

def solar_constraint2038(model,year,hour):
    return model.generators_dispatch["solar_PV",2038,hour] <= model.generators["solar_PV",2038]*Cf_solar.at[hour,0]
model.solar_constraint2038 = Constraint(years, hours, rule=solar_constraint2038)

def solar_constraint2041(model,year,hour):
    return model.generators_dispatch["solar_PV",2041,hour] <= model.generators["solar_PV",2041]*Cf_solar.at[hour,0]
model.solar_constraint2041 = Constraint(years, hours, rule=solar_constraint2041)

def solar_constraint2044(model,year,hour):
    return model.generators_dispatch["solar_PV",2044,hour] <= model.generators["solar_PV",2044]*Cf_solar.at[hour,0]
model.solar_constraint2044 = Constraint(years, hours, rule=solar_constraint2044)

def solar_constraint2047(model,year,hour):
    return model.generators_dispatch["solar_PV",2047,hour] <= model.generators["solar_PV",2047]*Cf_solar.at[hour,0]
model.solar_constraint2047 = Constraint(years, hours, rule=solar_constraint2047)

def solar_constraint2050(model,year,hour):
    return model.generators_dispatch["solar_PV",2050,hour] <= model.generators["solar_PV",2050]*Cf_solar.at[hour,0]
model.solar_constraint2050 = Constraint(years, hours, rule=solar_constraint2050)

#%% Onshore wind capacity constraints


def onshore_constraint2020(model,year,hour):
    return model.generators_dispatch["onshore_wind",2020,hour] <= model.generators["onshore_wind",2020]*Cf_onshore.at[hour,0]
model.onshore_constraint2020 = Constraint(years, hours, rule=onshore_constraint2020)

def onshore_constraint2023(model,year,hour):
    return model.generators_dispatch["onshore_wind",2023,hour] <= model.generators["onshore_wind",2023]*Cf_onshore.at[hour,0]
model.onshore_constraint2023 = Constraint(years, hours, rule=onshore_constraint2023)

def onshore_constraint2026(model,year,hour):
    return model.generators_dispatch["onshore_wind",2026,hour] <= model.generators["onshore_wind",2026]*Cf_onshore.at[hour,0]
model.onshore_constraint2026 = Constraint(years, hours, rule=onshore_constraint2026)

def onshore_constraint2029(model,year,hour):
    return model.generators_dispatch["onshore_wind",2029,hour] <= model.generators["onshore_wind",2029]*Cf_onshore.at[hour,0]
model.onshore_constraint2029 = Constraint(years, hours, rule=onshore_constraint2029)

def onshore_constraint2032(model,year,hour):
    return model.generators_dispatch["onshore_wind",2032,hour] <= model.generators["onshore_wind",2032]*Cf_onshore.at[hour,0]
model.onshore_constraint2032 = Constraint(years, hours, rule=onshore_constraint2032)

def onshore_constraint2035(model,year,hour):
    return model.generators_dispatch["onshore_wind",2035,hour] <= model.generators["onshore_wind",2035]*Cf_onshore.at[hour,0]
model.onshore_constraint2035 = Constraint(years, hours, rule=onshore_constraint2035)

def onshore_constraint2038(model,year,hour):
    return model.generators_dispatch["onshore_wind",2038,hour] <= model.generators["onshore_wind",2038]*Cf_onshore.at[hour,0]
model.onshore_constraint2038 = Constraint(years, hours, rule=onshore_constraint2038)

def onshore_constraint2041(model,year,hour):
    return model.generators_dispatch["onshore_wind",2041,hour] <= model.generators["onshore_wind",2041]*Cf_onshore.at[hour,0]
model.onshore_constraint2041 = Constraint(years, hours, rule=onshore_constraint2041)

def onshore_constraint2044(model,year,hour):
    return model.generators_dispatch["onshore_wind",2044,hour] <= model.generators["onshore_wind",2044]*Cf_onshore.at[hour,0]
model.onshore_constraint2044 = Constraint(years, hours, rule=onshore_constraint2044)

def onshore_constraint2047(model,year,hour):
    return model.generators_dispatch["onshore_wind",2047,hour] <= model.generators["onshore_wind",2047]*Cf_onshore.at[hour,0]
model.onshore_constraint2047 = Constraint(years, hours, rule=onshore_constraint2047)

def onshore_constraint2050(model,year,hour):
    return model.generators_dispatch["onshore_wind",2050,hour] <= model.generators["onshore_wind",2050]*Cf_onshore.at[hour,0]
model.onshore_constraint2050 = Constraint(years, hours, rule=onshore_constraint2050)

#%% Offshore wind capacity constraints


def offshore_constraint2020(model,year,hour):
    return model.generators_dispatch["offshore_wind",2020,hour] <= model.generators["offshore_wind",2020]*Cf_offshore.at[hour,0]
model.offshore_constraint2020 = Constraint(years, hours, rule=offshore_constraint2020)

def offshore_constraint2023(model,year,hour):
    return model.generators_dispatch["offshore_wind",2023,hour] <= model.generators["offshore_wind",2023]*Cf_offshore.at[hour,0]
model.offshore_constraint2023 = Constraint(years, hours, rule=offshore_constraint2023)

def offshore_constraint2026(model,year,hour):
    return model.generators_dispatch["offshore_wind",2026,hour] <= model.generators["offshore_wind",2026]*Cf_offshore.at[hour,0]
model.offshore_constraint2026 = Constraint(years, hours, rule=offshore_constraint2026)

def offshore_constraint2029(model,year,hour):
    return model.generators_dispatch["offshore_wind",2029,hour] <= model.generators["offshore_wind",2029]*Cf_offshore.at[hour,0]
model.offshore_constraint2029 = Constraint(years, hours, rule=offshore_constraint2029)

def offshore_constraint2032(model,year,hour):
    return model.generators_dispatch["offshore_wind",2032,hour] <= model.generators["offshore_wind",2032]*Cf_offshore.at[hour,0]
model.offshore_constraint2032 = Constraint(years, hours, rule=offshore_constraint2032)

def offshore_constraint2035(model,year,hour):
    return model.generators_dispatch["offshore_wind",2035,hour] <= model.generators["offshore_wind",2035]*Cf_offshore.at[hour,0]
model.offshore_constraint2035 = Constraint(years, hours, rule=offshore_constraint2035)

def offshore_constraint2038(model,year,hour):
    return model.generators_dispatch["offshore_wind",2038,hour] <= model.generators["offshore_wind",2038]*Cf_offshore.at[hour,0]
model.offshore_constraint2038 = Constraint(years, hours, rule=offshore_constraint2038)

def offshore_constraint2041(model,year,hour):
    return model.generators_dispatch["offshore_wind",2041,hour] <= model.generators["offshore_wind",2041]*Cf_offshore.at[hour,0]
model.offshore_constraint2041 = Constraint(years, hours, rule=offshore_constraint2041)

def offshore_constraint2044(model,year,hour):
    return model.generators_dispatch["offshore_wind",2044,hour] <= model.generators["offshore_wind",2044]*Cf_offshore.at[hour,0]
model.offshore_constraint2044 = Constraint(years, hours, rule=offshore_constraint2044)

def offshore_constraint2047(model,year,hour):
    return model.generators_dispatch["offshore_wind",2047,hour] <= model.generators["offshore_wind",2047]*Cf_offshore.at[hour,0]
model.offshore_constraint2047 = Constraint(years, hours, rule=offshore_constraint2047)

def offshore_constraint2050(model,year,hour):
    return model.generators_dispatch["offshore_wind",2050,hour] <= model.generators["offshore_wind",2050]*Cf_offshore.at[hour,0]
model.offshore_constraint2050 = Constraint(years, hours, rule=offshore_constraint2050)


#%% Storage

def storage__charge_constraint(model,tech,year,hour):
    return model.storage_charge['battery_store',year,hour] == sum(model.generators_dispatch[tech, year, hour] for tech in techs) - demand2w3h.at[hour,0]
model.storage__charge_constraint = Constraint(techs,years,hours,rule=storage__charge_constraint)

def stored_energy_constraint(model,year,hour):
    if hour == 0: 
        return model.stored_energy['battery_store',year,hour] == model.storage_charge['battery_store',year,hour] #- model.storage_discharge['battery_store',year,hour]
    else:
        return model.stored_energy['battery_store',year,hour] == model.stored_energy['battery_store',year,hour-1] + model.storage_charge['battery_store',year,hour] - model.storage_discharge['battery_store',year,hour]
model.stored_energy_constraint = Constraint(years,hours,rule=stored_energy_constraint)

def storage_constraint(model,year,hour):
    return model.storage_discharge['battery_store',year,hour] <= model.storage_charge['battery_store',year,hour] # GW
model.storage_constraint = Constraint(years, hours, rule=storage_constraint)

def storage_capacity_constraint(model,year,hour):
    return model.stored_energy['battery_store',year,hour] <= model.storage['battery_store',year] # GW
model.storage_capacity_constraint = Constraint(years,hours,rule=storage_capacity_constraint)

def build_years_storage(model,year):
    if year < years[0] + store_param.at["lifetime","battery_store"] - store_param.at["existing age","battery_store"]:
        constant = store_param.at["existing capacity","battery_store"] 
    else:
        constant = 0.
    
    return model.storage["battery_store",year] == constant + sum(model.storage_built["battery_store",yearb] for yearb in years if ((year>= yearb) and (year < yearb + store_param.at["lifetime","battery_store"]))) #GW
model.build_years_storage = Constraint(years, rule=build_years_storage)


def fixed_cost_constraint_storage(model,year):
    if store_param.at["learning parameter","battery_store"] == 0:
        return model.fixed_costs_storage["battery_store",year] == store_param.at["current capital cost","battery_store"]*1000 #EUR/GW
    else:
        return model.fixed_costs_storage["battery_store",year] == store_param.at["current capital cost","battery_store"]*1000 * (1+sum(model.storage_built["battery_store",yeart] for yeart in years if yeart < year))**(-store_param.at["learning parameter","battery_store"]) #EUR/GW
        # return model.fixed_costs["battery_store",year] == store_param.at["base cost","battery_store"] + (store_param.at["current capital cost","battery_store"]-store_param.at["base cost","battery_store"])*(1+sum(model.generators_built["battery_store",yeart] for yeart in years if yeart < year))**(-store_param.at["learning parameter","battery_store"])
model.fixed_cost_constraint_storage = Constraint(years, rule=fixed_cost_constraint_storage)

#%% Installed capacity constraints
# def generator_constraint(model,tech,year,hour):
#     return model.generators_dispatch[tech,year,hour] <= model.generators[tech,year] # GW
# model.generator_constraint = Constraint(techs, years, hours, rule=generator_constraint)

def generator_constraint2020(model,tech,hour):
    return model.generators_dispatch[tech,2020,hour] <= model.generators[tech,2020] # GW
model.generator_constraint2020 = Constraint(techs, hours, rule=generator_constraint2020)

def generator_constraint2023(model,tech,hour):
    return model.generators_dispatch[tech,2023,hour] <= model.generators[tech,2023] # GW
model.generator_constraint2023 = Constraint(techs, hours, rule=generator_constraint2023)

def generator_constraint2026(model,tech,hour):
    return model.generators_dispatch[tech,2026,hour] <= model.generators[tech,2026] # GW
model.generator_constraint2026 = Constraint(techs, hours, rule=generator_constraint2026)

def generator_constraint2029(model,tech,hour):
    return model.generators_dispatch[tech,2029,hour] <= model.generators[tech,2029] # GW
model.generator_constraint2029 = Constraint(techs, hours, rule=generator_constraint2029)

def generator_constraint2032(model,tech,hour):
    return model.generators_dispatch[tech,2032,hour] <= model.generators[tech,2032] # GW
model.generator_constraint2032 = Constraint(techs, hours, rule=generator_constraint2032)

def generator_constraint2035(model,tech,hour):
    return model.generators_dispatch[tech,2035,hour] <= model.generators[tech,2035] # GW
model.generator_constraint2035 = Constraint(techs, hours, rule=generator_constraint2035)

def generator_constraint2038(model,tech,hour):
    return model.generators_dispatch[tech,2038,hour] <= model.generators[tech,2038] # GW
model.generator_constraint2038 = Constraint(techs, hours, rule=generator_constraint2038)

def generator_constraint2041(model,tech,hour):
    return model.generators_dispatch[tech,2041,hour] <= model.generators[tech,2041] # GW
model.generator_constraint2041 = Constraint(techs, hours, rule=generator_constraint2041)

def generator_constraint2044(model,tech,hour):
    return model.generators_dispatch[tech,2044,hour] <= model.generators[tech,2044] # GW
model.generator_constraint2044 = Constraint(techs, hours, rule=generator_constraint2044)

def generator_constraint2047(model,tech,hour):
    return model.generators_dispatch[tech,2047,hour] <= model.generators[tech,2047] # GW
model.generator_constraint2047 = Constraint(techs, hours, rule=generator_constraint2047)

def generator_constraint2050(model,tech,hour):
    return model.generators_dispatch[tech,2050,hour] <= model.generators[tech,2050] # GW
model.generator_constraint2050 = Constraint(techs, hours, rule=generator_constraint2050)

def capacity_constraint(model,tech,year):
    return sum(model.generators[tech,year] for tech in techs) <= 1000000
model.capacity_constraint = Constraint(techs, years, rule=capacity_constraint)


def build_years(model,tech,year):
    if year < years[0] + parameters.at["lifetime",tech] - parameters.at["existing age",tech]:
        constant = parameters.at["existing capacity",tech] 
    else:
        constant = 0.
    
    return model.generators[tech,year] == constant + sum(model.generators_built[tech,yearb] for yearb in years if ((year>= yearb) and (year < yearb + parameters.at["lifetime",tech]))) #GW
model.build_years = Constraint(techs, years, rule=build_years)


def fixed_cost_constraint(model,tech,year):
    if parameters.at["learning parameter",tech] == 0:
        return model.fixed_costs[tech,year] == parameters.at["current capital cost",tech]*1000 #EUR/GW
    else:
        return model.fixed_costs[tech,year] == parameters.at["current capital cost",tech]*1000 * (1+sum(model.generators_built[tech,yeart] for yeart in years if yeart < year))**(-parameters.at["learning parameter",tech]) #EUR/GW
        # return model.fixed_costs[tech,year] == parameters.at["base cost",tech] + (parameters.at["current capital cost",tech]-parameters.at["base cost",tech])*(1+sum(model.generators_built[tech,yeart] for yeart in years if yeart < year))**(-parameters.at["learning parameter",tech])
model.fixed_cost_constraint = Constraint(techs, years, rule=fixed_cost_constraint)

#%% CO2 constraint
# Converting GW to MW (1000), 26 to go from 2 to 52 weeks, 3 to go from 1 to 3 years as that is the interval

def co2_constraint(model,tech,year,hour):
    return co2_budget >= sum((model.generators_dispatch[tech,year,hour] * 1000*26*3 * parameters.at["specific emissions",tech]) for tech in techs for hour in hours for year in years)
model.co2_constraint = Constraint(techs,years, hours,rule=co2_constraint)

# def co2_constraint2020(model,tech,hour):
#     return co2_budget/11 >= sum((model.generators_dispatch[tech,2020,hour] * parameters.at["specific emissions",tech]) for tech in techs for hour in hours) * 26*3
# model.co2_constraint2020 = Constraint(techs, hours,rule=co2_constraint2020)

# def co2_constraint2023(model,tech,hour):
#     return co2_budget/11 >= sum((model.generators_dispatch[tech,2023,hour] * parameters.at["specific emissions",tech]) for tech in techs for hour in hours)* 26*3
# model.co2_constraint2023 = Constraint(techs, hours,rule=co2_constraint2023)

# def co2_constraint2026(model,tech,hour):
#     return co2_budget/11 >= sum((model.generators_dispatch[tech,2026,hour] * parameters.at["specific emissions",tech]) for tech in techs for hour in hours)* 26*3
# model.co2_constraint2026 = Constraint(techs, hours,rule=co2_constraint2026)

# def co2_constraint2029(model,tech,hour):
#     return co2_budget/11 >= sum((model.generators_dispatch[tech,2029,hour] * parameters.at["specific emissions",tech]) for tech in techs for hour in hours)* 26*3
# model.co2_constraint2029 = Constraint(techs, hours,rule=co2_constraint2029)

# def co2_constraint2032(model,tech,hour):
#     return co2_budget/11 >= sum((model.generators_dispatch[tech,2032,hour] *  parameters.at["specific emissions",tech]) for tech in techs for hour in hours)* 26*3
# model.co2_constraint2032 = Constraint(techs, hours,rule=co2_constraint2032)

# def co2_constraint2035(model,tech,hour):
#     return co2_budget/11 >= sum((model.generators_dispatch[tech,2035,hour] * parameters.at["specific emissions",tech]) for tech in techs for hour in hours)* 26*3
# model.co2_constraint2035 = Constraint(techs, hours,rule=co2_constraint2035)

# def co2_constraint2038(model,tech,hour):
#     return co2_budget/11 >= sum((model.generators_dispatch[tech,2038,hour] * parameters.at["specific emissions",tech]) for tech in techs for hour in hours)* 26*3
# model.co2_constraint2038 = Constraint(techs, hours,rule=co2_constraint2038)

# def co2_constraint2041(model,tech,hour):
#     return co2_budget/11 >= sum((model.generators_dispatch[tech,2041,hour] * parameters.at["specific emissions",tech]) for tech in techs for hour in hours)* 26*3
# model.co2_constraint2041 = Constraint(techs, hours,rule=co2_constraint2041)

# def co2_constraint2044(model,tech,hour):
#     return co2_budget/11 >= sum((model.generators_dispatch[tech,2044,hour] * parameters.at["specific emissions",tech]) for tech in techs for hour in hours)* 26*3
# model.co2_constraint2044 = Constraint(techs, hours,rule=co2_constraint2044)

# def co2_constraint2047(model,tech,hour):
#     return co2_budget/11 >= sum((model.generators_dispatch[tech,2047,hour] * parameters.at["specific emissions",tech]) for tech in techs for hour in hours)* 26*3
# model.co2_constraint2047 = Constraint(techs, hours,rule=co2_constraint2047)

def co2_constraint2050(model,tech,hour):
    return 0 >= sum((model.generators_dispatch[tech,2050,hour] *1000* parameters.at["specific emissions",tech]) for tech in techs for hour in hours)
model.co2_constraint2050 = Constraint(techs, hours,rule=co2_constraint2050)


executionTime = (time.time() - startTime)
print('Writing time for Pyomo model in seconds: ' + str(executionTime))

Cost of existing capacity = 0.00
Writing time for Pyomo model in seconds: 649.9915118217468


## GLPK installation

Keywords: GLPK

[GLPK](https://en.wikibooks.org/wiki/GLPK) is a the open-source **G**NU **L**inear **P**rogramming **K**it available for use under the GNU General Public License 3. GLPK is a single-threaded simplex solver generally suited to small to medium scale linear-integer programming problems. It is written in C with minimal dependencies and is therefore highly portable among computers and operating systems. GLPK is often 'good enough' for many examples. For larger problems users should consider higher-performance solvers, such as COIN-OR CBC, that can take advantage of multi-threaded processors.

In [ ]:
!apt-get install -y -qq glpk-utils

In [ ]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('Demand  = ', model.demand())
print('Labor A = ', model.laborA())
print('Labor B = ', model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2600.0
  Upper bound: 2600.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 6
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.020076274871826172
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

## COIN-OR CBC installation

Keywords: cbc installation

[COIN-OR CBC](https://github.com/coin-or/Cbc) is a multi-threaded open-source **C**oin-or **b**ranch and **c**ut mixed-integer linear programming solver written in C++ under the Eclipse Public License (EPL). CBC is generally a good choice for a general purpose MILP solver for medium to large scale problems.

In [ ]:
!apt-get install -y -qq coinor-cbc

In [ ]:
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('Demand  = ', model.demand())
print('Labor A = ', model.laborA())
print('Labor B = ', model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -2600.0
  Upper bound: -2600.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 6
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  Termination condition: optimal
  Error rc: 0
  Time: 0.019547700881958008
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


## COIN-OR Ipopt installation

Keywords: Ipopt installation

[COIN-OR Ipopt](https://github.com/coin-or/Ipopt) is an open-source **I**nterior **P**oint **Opt**imizer for large-scale nonlinear optimization available under the Eclipse Public License (EPL). It is well-suited to solving nonlinear programming problems without integer or binary constraints.

In [ ]:
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [ ]:
SolverFactory('ipopt', executable='/content/ipopt').solve(model).write()

# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('Demand  = ', model.demand())
print('Labor A = ', model.laborA())
print('Labor B = ', model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.8\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.022800445556640625
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit

## COIN-OR Bonmin installation

[COIN-OR Bonmin](https://www.coin-or.org/Bonmin/Intro.html) is a **b**asic **o**pen-source solver for **n**onlinear **m**ixed-**in**teger programming problems (MINLP). It utilizes CBC and Ipopt for solving relaxed subproblems.

In [1]:
!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
!unzip -o -q bonmin-linux64

In [ ]:
SolverFactory('bonmin', executable='/content/bonmin').solve(model).write()

# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('Demand  = ', model.demand())
print('Labor A = ', model.laborA())
print('Labor B = ', model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 0.025995254516601562
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit =  2600.000025999979

## COIN-OR Couenne installation

[COIN-OR Couenne](https://www.coin-or.org/Couenne/)  is attempts to find global optima for mixed-integer nonlinear programming problems (MINLP).

In [8]:
!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
!unzip -o -q couenne-linux64

zsh:1: command not found: wget
unzip:  cannot find or open couenne-linux64, couenne-linux64.zip or couenne-linux64.ZIP.


In [9]:
SolverFactory('couenne', executable='/content/couenne').solve(model).write()

# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('Demand  = ', model.demand())
print('Labor A = ', model.laborA())
print('Labor B = ', model.laborB())

    for solver asl. File with name=/content/couenne either does not exist or
    it is not executable. To skip this validation, call set_executable with
    validate=False.
Traceback (most recent call last):
  File "/Users/frederikmelson/opt/anaconda3/lib/python3.9/site-packages/pyomo/opt/base/solvers.py", line 165, in __call__
    opt = self._cls[_implicit_solvers[mode]](**kwds)
  File "/Users/frederikmelson/opt/anaconda3/lib/python3.9/site-packages/pyomo/solvers/plugins/solvers/ASL.py", line 43, in __init__
    SystemCallSolver.__init__(self, **kwds)
  File "/Users/frederikmelson/opt/anaconda3/lib/python3.9/site-packages/pyomo/opt/solver/shellcmd.py", line 55, in __init__
    self.set_executable(name=executable, validate=validate)
  File "/Users/frederikmelson/opt/anaconda3/lib/python3.9/site-packages/pyomo/opt/solver/shellcmd.py", line 100, in set_executable
    raise ValueError(
ValueError: Failed to set executable for solver asl. File with name=/content/couenne either does not exi

RuntimeError: Attempting to use an unavailable solver.

The SolverFactory was unable to create the solver "couenne"
and returned an UnknownSolver object.  This error is raised at the point
where the UnknownSolver object was used as if it were valid (by calling
method "solve").

The original solver was created with the following parameters:
	executable: /content/couenne
	type: couenne
	_args: ()
	options: {}

## Gecode installation

Keywords: Gecode installation

In [ ]:
!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
!unzip -o -q gecode-linux64

Gecode solves constraint programming problems and does not support continuous variables. We therefore create a second model using exclusively discrete variables.

In [ ]:
from pyomo.environ import *

# create a model
discrete_model = ConcreteModel()

# declare decision variables
discrete_model.x = Var(domain=NonNegativeIntegers)
discrete_model.y = Var(domain=NonNegativeIntegers)

# declare objective
discrete_model.profit = Objective(expr = 40*discrete_model.x + 30*discrete_model.y, sense=maximize)

# declare constraints
discrete_model.demand = Constraint(expr = discrete_model.x <= 40)
discrete_model.laborA = Constraint(expr = discrete_model.x + discrete_model.y <= 80)
discrete_model.laborB = Constraint(expr = 2*discrete_model.x + discrete_model.y <= 100)

discrete_model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x + 30*y

3 Constraint Declarations
    demand : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :    x :  40.0 :   True
    laborA : Size=1, Index=None, Active=True
        Key  : Lower : Body  : Upper : Active
        None :  -Inf : x + y :  80.0 :   True
    laborB : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y : 100.0 :   True

6 Declarations: x y profit demand laborA laborB


In [ ]:
SolverFactory('gecode', executable='/content/gecode').solve(discrete_model).write()

# display solution
print('\nProfit = ', discrete_model.profit())

print('\nDecision Variables')
print('x = ', discrete_model.x())
print('y = ', discrete_model.y())

print('\nConstraints')
print('Demand  = ', discrete_model.demand())
print('Labor A = ', discrete_model.laborA())
print('Labor B = ', discrete_model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: gecode 4.4.0\x3a optimal solution; 201 nodes, 0 fails, objective 2600
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.019869565963745117
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of 

<!--NAVIGATION-->
< [Installing a Pyomo/Python Development Environment](http://nbviewer.jupyter.org/github/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/01.01-Installing-Pyomo.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Running Pyomo on the Notre Dame CRC Cluster](http://nbviewer.jupyter.org/github/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/01.03-Running-Pyomo-on-the-Notre-Dame-CRC-Cluster.ipynb) ><p><a href="https://colab.research.google.com/github/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/01.02-Running-Pyomo-on-Google-Colab.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>